# Reading multiple csv files in PyTorch
Author: [Biswajit Sahoo](https://biswajitsahoo1111.github.io/)

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a href="https://colab.research.google.com/github/biswajitsahoo1111/blog_notebooks/blob/master/Reading_multiple_csv_files_in_PyTorch.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a href="https://github.com/biswajitsahoo1111/blog_notebooks/blob/master/Reading_multiple_csv_files_in_PyTorch.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://www.dropbox.com/s/qxz8zmctpopulck/Reading_multiple_csv_files_in_PyTorch.ipynb?dl=1"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In many engineering applications data are usually stored in CSV (Comma Separated Values) files. In big data applications, it's not uncommon to obtain thousands of csv files. As the number of files increases, at some point, we can no longer load the whole dataset into computer's memory. In deep learning applications it is increasingly common to come across datasets that don't fit in the computer's memory. In that case, we have to devise a way so as to be able to read chunks of data at a time so that the model can be trained using the whole dataset.

There are many ways to achieve this objective. In this post, we will adopt an approach that allows us to read csv files in chunks and preprocess those files in whatever way we like. Then we can pass the processed data to train any deep learning model. Though we will use csv files in this post, the method is general enough to work for other file formats (such as .txt, .npz, etc.) as well. We will demonstrate the procedure using 500 csv files. But the method can be easily extended to huge datasets involving thousands of csv files. 

This post is self-sufficient in the sense that readers don’t have to download any data from anywhere. Just run the following codes sequentially. First, a folder named “random_data” will be created in current working directory and .csv files will be saved in it. Subsequently, files will be read from that folder and processed. Just make sure that your current working directory doesn’t have an old folder named “random_data”. Then run the following code cells. We will use PyTorch to run our deep learning model. For efficiency in data loading, we will use PyTorch dataloaders.

## Outline:
1. [Create 500 ".csv" files and save it in the folder “random_data” in current working directory.](#sec_1)
2. [Create a custom dataloader.](#sec_2)
3. [Feed the chunks of data to a CNN model and train it for several epochs.](#sec_3)
4. [Make prediction on new data for which labels are not knownn.](#sec_4)

<a id="sec_1"></a>

## 1. Create 500 .csv files of random data

As we intend to train a CNN model for classification using our data, we will generate data for 5 different classes. The dataset that we will create is a contrived one. But readers can modify the approach slightly to cater to their need. Following is the process that we will follow.

* Each .csv file will have one column of data with 1024 entries.
* Each file will be saved using one of the following names (Fault_1, Fault_2, Fault_3, Fault_4, Fault_5). The dataset is balanced, meaning, for each category, we have approximately same number of observations. Data files in "Fault_1" category will have names as "Fault_1_001.csv", "Fault_1_002.csv", "Fault_1_003.csv", ..., "Fault_1_100.csv". Similarly for other classes.

In [1]:
import numpy as np
import os
import glob
np.random.seed(1111)

First create a function that will generate random files.

In [2]:
def create_random_csv_files(fault_classes, number_of_files_in_each_class):
    os.mkdir("./random_data/")  # Make a directory to save created files.
    for fault_class in fault_classes:
        for i in range(number_of_files_in_each_class):
            data = np.random.rand(1024,)
            file_name = "./random_data/" + eval("fault_class") + "_" + "{0:03}".format(i+1) + ".csv" # This creates file_name
            np.savetxt(eval("file_name"), data, delimiter = ",", header = "V1", comments = "")
        print(str(eval("number_of_files_in_each_class")) + " " + eval("fault_class") + " files"  + " created.")

Now use the function to create 100 files each for five fault types.

In [3]:
create_random_csv_files(["Fault_1", "Fault_2", "Fault_3", "Fault_4", "Fault_5"], number_of_files_in_each_class = 100)

100 Fault_1 files created.
100 Fault_2 files created.
100 Fault_3 files created.
100 Fault_4 files created.
100 Fault_5 files created.


In [4]:
files = glob.glob("./random_data/*")
print("Total number of files: ", len(files))
print("Showing first 10 files...")
files[:10]

Total number of files:  500
Showing first 10 files...


['./random_data\\Fault_1_001.csv',
 './random_data\\Fault_1_002.csv',
 './random_data\\Fault_1_003.csv',
 './random_data\\Fault_1_004.csv',
 './random_data\\Fault_1_005.csv',
 './random_data\\Fault_1_006.csv',
 './random_data\\Fault_1_007.csv',
 './random_data\\Fault_1_008.csv',
 './random_data\\Fault_1_009.csv',
 './random_data\\Fault_1_010.csv']

To extract labels from file name, extract the part of the file name that corresponds to fault type.

In [5]:
print(files[0])

./random_data\Fault_1_001.csv


In [6]:
print(files[0][14:21])

Fault_1


Now that data have been created, we will go to the next step. That is, create a custom dataloader, preprocess the time series like data into a matrix like shape such that a 2-D CNN can ingest it. We reshape the data in that way to just illustrate the point. Readers should use their own preprocessing steps.

<a id="sec_2"></a>

## 2. Write a custom dataloader

We have to first create a `Dataset` class. Then we can pass the dataset to the dataloader. Every dataset class must implement the `__len__` method that determines the length of the dataset and `__getitem__` method that iterates over the dataset item by item. In our case, item would mean the processed version of a chunk of data.

The following dataset class takes a list of file names as first argument. The second argument is batch_size. batch_size determines how many files we will process at one go. As we will be solving a classification problem, we have to assign labels to each raw data. We will use following labels for convenience.

|Class|Label|
|:----:|:----:|
|Fault_1|0|
|Fault_2|1|
|Fault_3|2|
|Fault_4|3|
|Fault_5|4|

In [7]:
import pandas as pd
import re   
import torch
from torch.utils.data import Dataset
print("PyTorch Version: ", torch.__version__)

PyTorch Version:  1.7.1


In [8]:
class CustomDataset(Dataset):
  def __init__(self, filenames, batch_size):
    # `filenames` is a list of strings the contains all file names.
    # `batch_size` is the determines the number of files that we want to read in a chunk.
        self.filenames= filenames
        self.batch_size = batch_size
  def __len__(self):
        return int(np.ceil(len(self.filenames) / float(self.batch_size)))   # Number of chunks.
  def __getitem__(self, idx): #idx means index of the chunk.
    # In this method, we do all the preprocessing.
    # First read data from files in a chunk. Preprocess it. Extract labels. Then return data and labels.
        batch_x = self.filenames[idx * self.batch_size:(idx + 1) * self.batch_size]   # This extracts one batch of file names from the list `filenames`.
        data = []
        labels = []
        label_classes = ["Fault_1", "Fault_2", "Fault_3", "Fault_4", "Fault_5"]
        for file in batch_x:
            temp = pd.read_csv(open(file,'r')) # Change this line to read any other type of file
            data.append(temp.values.reshape(32,32,1)) # Convert column data to matrix like data with one channel
            pattern = "^" + eval("file[14:21]")      # Pattern extracted from file_name
            for j in range(len(label_classes)):
                if re.match(pattern, label_classes[j]): # Pattern is matched against different label_classes
                    labels.append(j)  
        data = np.asarray(data).reshape(-1,1,32,32) # Because of Pytorch's channel first convention
        labels = np.asarray(labels)

        # The following condition is actually needed in Pytorch. Otherwise, for our particular example, the iterator will be an infinite loop.
        # Readers can verify this by removing this condition.
        if idx == self.__len__():  
          raise IndexError

        return data, labels

To read any other file format, inside the `__getitem__` method change the line that reads files. This will enable us to read different file formats, be it `.txt` or `.npz` or any other. Preprocessing of data, different from what we have done in this blog, can be done within the `__getitem__` method.

Now we will check whether the dataset works as intended or not. We will set batch_size to 10. This means that files in chunks of 10 will be read and processed. The list of files from which 10 are chosen can be an ordered file list or shuffled list. In case, the files are not shuffled, use np.random.shuffle(file_list) to shuffle files.

In the demonstration, we will read files from an ordered list. This will help us check any errors in the code.

In [9]:
check_dataset = CustomDataset(filenames = files, batch_size = 10)

In [10]:
check_dataset.__len__()

50

In [11]:
for i, (data, labels) in enumerate(check_dataset):
  print(data.shape, labels.shape)
  print(labels)
  if i == 5: break

(10, 1, 32, 32) (10,)
[0 0 0 0 0 0 0 0 0 0]
(10, 1, 32, 32) (10,)
[0 0 0 0 0 0 0 0 0 0]
(10, 1, 32, 32) (10,)
[0 0 0 0 0 0 0 0 0 0]
(10, 1, 32, 32) (10,)
[0 0 0 0 0 0 0 0 0 0]
(10, 1, 32, 32) (10,)
[0 0 0 0 0 0 0 0 0 0]
(10, 1, 32, 32) (10,)
[0 0 0 0 0 0 0 0 0 0]


Run the above cell multiple times to observe different labels. Label 1 appears only when all the files corresponding to “Fault_1” have been read. There are 100 files for “Fault_1” and we have set batch_size to 10. In the above cell we are iterating over the generator only 6 times. When number of iterations become greater than 10, we see label 1 and subsequently other labels. This will happen only if our initial file list is not shuffled. If the original list is shuffled, we will get random labels.

To train a deep learning model, we need to create a data loader from the dataset. Dataloaders offer multi-worker, multi-processing capabilities without requiring us to right codes for that. So let's first create a dataloader from the dataset.

In [12]:
from torch.utils.data import DataLoader

In [13]:
dataloader = DataLoader(check_dataset,batch_size = None, shuffle = True) # Here we select batch size to be None as we have already batched our data in dataset.

Check whether dataloader works on not.

In [14]:
for i, (data,labels) in enumerate(dataloader):
    print(data.shape, labels.shape)
    print(labels)  # Just to see the labels.
    if i == 3: break

torch.Size([10, 1, 32, 32]) torch.Size([10])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.int32)
torch.Size([10, 1, 32, 32]) torch.Size([10])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.int32)
torch.Size([10, 1, 32, 32]) torch.Size([10])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.int32)
torch.Size([10, 1, 32, 32]) torch.Size([10])
tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4], dtype=torch.int32)


Now that dataloader works, we will use it to train a simple deep learning model. The focus of this post is not on the model itself. So we will use a simplest model. If readers want a different model, they can do so by just replacing our model with theirs.

<a id="sec_3"></a>

## 3. Feed chunks of data to a CNN model and train it for several epochs

But before we build the model and train it, we will first move our files to different folders depending on their fault type. We do so as it will be convenient for later to create a training, validation, and test set from the data.

In [15]:
import shutil

Create five different folders one each for a given fault type.

In [16]:
fault_folders = ["Fault_1", "Fault_2", "Fault_3", "Fault_4", "Fault_5"]
for folder_name in fault_folders:
    os.mkdir(os.path.join("./random_data", folder_name))

Move files into those folders.

In [17]:
for file in files:
    pattern = "^" + eval("file[14:21]")
    for j in range(len(fault_folders)):
        if re.match(pattern, fault_folders[j]):
            dest = os.path.join("./random_data/",eval("fault_folders[j]"))
            shutil.move(file, dest)

In [18]:
glob.glob("./random_data/*")

['./random_data\\Fault_1',
 './random_data\\Fault_2',
 './random_data\\Fault_3',
 './random_data\\Fault_4',
 './random_data\\Fault_5']

In [19]:
glob.glob("./random_data/Fault_1/*")[:10] # Showing first 10 files of Fault_1 folder

['./random_data/Fault_1\\Fault_1_001.csv',
 './random_data/Fault_1\\Fault_1_002.csv',
 './random_data/Fault_1\\Fault_1_003.csv',
 './random_data/Fault_1\\Fault_1_004.csv',
 './random_data/Fault_1\\Fault_1_005.csv',
 './random_data/Fault_1\\Fault_1_006.csv',
 './random_data/Fault_1\\Fault_1_007.csv',
 './random_data/Fault_1\\Fault_1_008.csv',
 './random_data/Fault_1\\Fault_1_009.csv',
 './random_data/Fault_1\\Fault_1_010.csv']

In [20]:
glob.glob("./random_data/Fault_3/*")[:10] # Showing first 10 files of Fault_3 folder

['./random_data/Fault_3\\Fault_3_001.csv',
 './random_data/Fault_3\\Fault_3_002.csv',
 './random_data/Fault_3\\Fault_3_003.csv',
 './random_data/Fault_3\\Fault_3_004.csv',
 './random_data/Fault_3\\Fault_3_005.csv',
 './random_data/Fault_3\\Fault_3_006.csv',
 './random_data/Fault_3\\Fault_3_007.csv',
 './random_data/Fault_3\\Fault_3_008.csv',
 './random_data/Fault_3\\Fault_3_009.csv',
 './random_data/Fault_3\\Fault_3_010.csv']

Prepare the data for training set, validation set, and test_set. For each fault type, we will keep 70 files for training, 10 files for validation and 20 files for testing.

In [21]:
fault_1_files = glob.glob("./random_data/Fault_1/*")
fault_2_files = glob.glob("./random_data/Fault_2/*")
fault_3_files = glob.glob("./random_data/Fault_3/*")
fault_4_files = glob.glob("./random_data/Fault_4/*")
fault_5_files = glob.glob("./random_data/Fault_5/*")

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
fault_1_train, fault_1_test = train_test_split(fault_1_files, test_size = 20, random_state = 5)
fault_2_train, fault_2_test = train_test_split(fault_2_files, test_size = 20, random_state = 54)
fault_3_train, fault_3_test = train_test_split(fault_3_files, test_size = 20, random_state = 543)
fault_4_train, fault_4_test = train_test_split(fault_4_files, test_size = 20, random_state = 5432)
fault_5_train, fault_5_test = train_test_split(fault_5_files, test_size = 20, random_state = 54321)

In [24]:
fault_1_train, fault_1_val = train_test_split(fault_1_train, test_size = 10, random_state = 1)
fault_2_train, fault_2_val = train_test_split(fault_2_train, test_size = 10, random_state = 12)
fault_3_train, fault_3_val = train_test_split(fault_3_train, test_size = 10, random_state = 123)
fault_4_train, fault_4_val = train_test_split(fault_4_train, test_size = 10, random_state = 1234)
fault_5_train, fault_5_val = train_test_split(fault_5_train, test_size = 10, random_state = 12345)

In [25]:
train_file_names = fault_1_train + fault_2_train + fault_3_train + fault_4_train + fault_5_train
validation_file_names = fault_1_val + fault_2_val + fault_3_val + fault_4_val + fault_5_val
test_file_names = fault_1_test + fault_2_test + fault_3_test + fault_4_test + fault_5_test

# Shuffle training files (We don't need to shuffle validation and test data)
np.random.shuffle(train_file_names)

In [26]:
print("Number of train_files:" ,len(train_file_names))
print("Number of validation_files:" ,len(validation_file_names))
print("Number of test_files:" ,len(test_file_names))

Number of train_files: 350
Number of validation_files: 50
Number of test_files: 100


Create the datasets and dataloaders for training, validation, and test set.

In [27]:
batch_size = 10
train_dataset = CustomDataset(filenames = train_file_names, batch_size = batch_size)
val_dataset = CustomDataset(filenames = validation_file_names, batch_size = batch_size)
test_dataset = CustomDataset(filenames = test_file_names, batch_size = batch_size)

train_dataloader = DataLoader(train_dataset, batch_size = None, shuffle = True)
val_dataloader = DataLoader(val_dataset, batch_size = None)  # Shuffle is False by default.
test_dataloader = DataLoader(test_dataset, batch_size = None)

Now create the model. We will build one of the simplest models. Readers are free to choose a different model of their choice. If `torchsummary` is not installed, use `pip install torchsummary` to install it.

In [28]:
from torch.nn import Sequential, Conv2d, MaxPool2d, Flatten, Linear, ReLU, Softmax
from torchsummary import summary

In [29]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [30]:
model = Sequential(
        Conv2d(in_channels = 1, out_channels = 16, kernel_size = 3),
        ReLU(),
        MaxPool2d(2),
        Conv2d(16,32,3),
        ReLU(),
        MaxPool2d(2),
        Flatten(),
        Linear(in_features = 1152, out_features=16),
        ReLU(),
        Linear(16, 5),
        Softmax(dim = 1)
)
model.to(dev)
summary(model,input_size = (1,32,32), device = dev.type)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 30, 30]             160
              ReLU-2           [-1, 16, 30, 30]               0
         MaxPool2d-3           [-1, 16, 15, 15]               0
            Conv2d-4           [-1, 32, 13, 13]           4,640
              ReLU-5           [-1, 32, 13, 13]               0
         MaxPool2d-6             [-1, 32, 6, 6]               0
           Flatten-7                 [-1, 1152]               0
            Linear-8                   [-1, 16]          18,448
              ReLU-9                   [-1, 16]               0
           Linear-10                    [-1, 5]              85
          Softmax-11                    [-1, 5]               0
Total params: 23,333
Trainable params: 23,333
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

In [31]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [32]:
model = model.float()   # We will make all model parameters floats.
epochs = 10
for epoch in range(epochs):
    running_loss_train = 0.0
    running_loss_val = 0.0
    correct_train = 0.0
    correct_val = 0.0
    num_labels_train = 0.0
    num_labels_val = 0.0

    # Training loop
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(dev), labels.type(torch.LongTensor).to(dev) # PyTorch expects categorical targets as LongTensor.
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss_train = running_loss_train + loss.item()
        correct_train = correct_train + (torch.argmax(outputs,dim = 1) == labels).float().sum()
        num_labels_train = num_labels_train + len(labels)

    # Validation loop
    for inputs, labels in val_dataloader:
        inputs, labels = inputs.to(dev), labels.type(torch.LongTensor).to(dev) # PyTorch expects categorical targets as LongTensor.
        outputs = model(inputs.float())
        loss = criterion(outputs, labels)
        running_loss_val = running_loss_val + loss.item()
        correct_val = correct_val + (torch.argmax(outputs, dim = 1) == labels).float().sum()
        num_labels_val = num_labels_val + len(labels)

    train_accuracy = correct_train/num_labels_train
    val_accuracy = correct_val/num_labels_val
    print("Epoch:{}, Train_loss: {:.4f}, Train_accuracy: {:.4f}, Val_loss: {:.4f}, Val_accuracy: {:.4f}".\
        format(epoch, running_loss_train/len(train_dataloader), train_accuracy, running_loss_val/len(val_dataloader), val_accuracy))

Epoch:0, Train_loss: 1.6115, Train_accuracy: 0.1829, Val_loss: 1.6098, Val_accuracy: 0.2000
Epoch:1, Train_loss: 1.6099, Train_accuracy: 0.2000, Val_loss: 1.6098, Val_accuracy: 0.2000
Epoch:2, Train_loss: 1.6098, Train_accuracy: 0.2000, Val_loss: 1.6098, Val_accuracy: 0.2000
Epoch:3, Train_loss: 1.6098, Train_accuracy: 0.2000, Val_loss: 1.6098, Val_accuracy: 0.2000
Epoch:4, Train_loss: 1.6097, Train_accuracy: 0.2000, Val_loss: 1.6098, Val_accuracy: 0.2000
Epoch:5, Train_loss: 1.6096, Train_accuracy: 0.2000, Val_loss: 1.6098, Val_accuracy: 0.2000
Epoch:6, Train_loss: 1.6096, Train_accuracy: 0.2000, Val_loss: 1.6098, Val_accuracy: 0.2000
Epoch:7, Train_loss: 1.6094, Train_accuracy: 0.2000, Val_loss: 1.6098, Val_accuracy: 0.2000
Epoch:8, Train_loss: 1.6092, Train_accuracy: 0.2000, Val_loss: 1.6098, Val_accuracy: 0.2000
Epoch:9, Train_loss: 1.6088, Train_accuracy: 0.2000, Val_loss: 1.6098, Val_accuracy: 0.2000


Before we make any comments on training accuracy and validation accuracy, we should keep in mind that our original dataset contains only random numbers. So it would be better if we don't interpret the results here.

Compute test score.

In [33]:
running_loss_test = 0.0
correct_test = 0.0
num_labels_test = 0.0
for inputs, labels in test_dataloader:
    inputs, labels = inputs.to(dev), labels.type(torch.LongTensor).to(dev) # Pytorch expects categorical targets as LongTensor.
    outputs = model(inputs.float())
    loss = criterion(outputs.to(dev), labels.to(dev))
    running_loss_test = running_loss_test + loss.item()
    correct_test = correct_test + (torch.argmax(outputs, dim = 1) == labels).float().sum()
    num_labels_test = num_labels_test + len(labels)

test_accuracy = correct_test/num_labels_test
print("Test_loss: {:.4f}, Test_accuracy: {:.4f}".format(running_loss_test/len(test_dataloader), test_accuracy))

Test_loss: 1.6098, Test_accuracy: 0.2000


<a id="sec_4"></a>

## 4. How to make predictions?

Until now, we have evaluated our model on a kept out test set. For our test set, both data and labels were known. So we evaluated its performance. But oftentimes, for test set, we don’t have access to true labels. Rather, we have to make predictions on the data available. This is the case in online competitions where we have to submit our predictions on a test set for which we don’t know the labels. We will call this set (without any labels) the prediction set. This naming convention is arbitrary but we will stick with it.

If the whole of our prediction set fits into memory, we can just make prediction on this data and then use `np.argmax()` or `torch.argmax()` to obtain predicted class labels. Otherwise, we can read files in prediction set in chunks, make predictions on the chunks and finally append our result.

Yet another pedantic way of doing this is to write a separate dataset to read files from the prediction set in chunks and make predictions on it. We will show how this approach works. As we don’t have a prediction set yet, we will first create some files and save it to the prediction set.

In [34]:
def create_prediction_set(num_files = 20):
    os.mkdir("./random_data/prediction_set")
    for i in range(num_files):
        data = np.random.randn(1024,)
        file_name = "./random_data/prediction_set/"  + "file_" + "{0:03}".format(i+1) + ".csv" # This creates file_name
        np.savetxt(eval("file_name"), data, delimiter = ",", header = "V1", comments = "")
    print(str(eval("num_files")) + " "+ " files created in prediction set.")

Create some files for prediction set.

In [35]:
create_prediction_set(num_files = 55)

55  files created in prediction set.


In [36]:
prediction_files = glob.glob("./random_data/prediction_set/*")
print("Total number of files: ", len(prediction_files))
print("Showing first 10 files...")
prediction_files[:10]

Total number of files:  55
Showing first 10 files...


['./random_data/prediction_set\\file_001.csv',
 './random_data/prediction_set\\file_002.csv',
 './random_data/prediction_set\\file_003.csv',
 './random_data/prediction_set\\file_004.csv',
 './random_data/prediction_set\\file_005.csv',
 './random_data/prediction_set\\file_006.csv',
 './random_data/prediction_set\\file_007.csv',
 './random_data/prediction_set\\file_008.csv',
 './random_data/prediction_set\\file_009.csv',
 './random_data/prediction_set\\file_010.csv']

The prediction dataset will be slightly different from our previous custom dataset class. We only need to return data in this case.

In [37]:
class PredictionDataset(Dataset):
  def __init__(self, filenames, batch_size):
        self.filenames= filenames
        self.batch_size = batch_size
  def __len__(self):
        return int(np.ceil(len(self.filenames) / float(self.batch_size)))
  def __getitem__(self, idx):
        batch_x = self.filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        data = []
        labels = []
        label_classes = ["Fault_1", "Fault_2", "Fault_3", "Fault_4", "Fault_5"]
        for file in batch_x:
            temp = pd.read_csv(open(file,'r')) 
            data.append(temp.values.reshape(32,32,1)) 
        data = np.asarray(data).reshape(-1,1,32,32) 
        
        
        if idx == self.__len__():  
          raise IndexError

        return data

Check whether the dataset and dataloader work or not.

In [38]:
prediction_dataset = PredictionDataset(prediction_files, batch_size = 10)
prediction_dataloader = DataLoader(prediction_dataset,batch_size = None, shuffle = False)

In [39]:
for data in prediction_dataloader:
    print(data.shape)

torch.Size([10, 1, 32, 32])
torch.Size([10, 1, 32, 32])
torch.Size([10, 1, 32, 32])
torch.Size([10, 1, 32, 32])
torch.Size([10, 1, 32, 32])
torch.Size([5, 1, 32, 32])


Make predictions.

In [40]:
preds = []
for data in prediction_dataloader:
    data = data.to(dev)
    preds.append(model(data.float()))
preds = torch.cat(preds)
preds

tensor([[0.1909, 0.2369, 0.1795, 0.2210, 0.1718],
        [0.1997, 0.2257, 0.1775, 0.2262, 0.1709],
        [0.1945, 0.2329, 0.1789, 0.2226, 0.1711],
        [0.1951, 0.2318, 0.1789, 0.2231, 0.1711],
        [0.1923, 0.2352, 0.1793, 0.2217, 0.1716],
        [0.1939, 0.2332, 0.1790, 0.2225, 0.1713],
        [0.1959, 0.2307, 0.1787, 0.2236, 0.1711],
        [0.1966, 0.2300, 0.1787, 0.2238, 0.1708],
        [0.1992, 0.2268, 0.1781, 0.2253, 0.1706],
        [0.1930, 0.2335, 0.1781, 0.2231, 0.1722],
        [0.1947, 0.2323, 0.1790, 0.2229, 0.1712],
        [0.1886, 0.2396, 0.1797, 0.2199, 0.1721],
        [0.1879, 0.2406, 0.1798, 0.2195, 0.1722],
        [0.2032, 0.2211, 0.1762, 0.2286, 0.1709],
        [0.1843, 0.2443, 0.1792, 0.2187, 0.1735],
        [0.2028, 0.2228, 0.1779, 0.2267, 0.1698],
        [0.1916, 0.2360, 0.1794, 0.2214, 0.1717],
        [0.2079, 0.2171, 0.1772, 0.2290, 0.1689],
        [0.1945, 0.2325, 0.1790, 0.2228, 0.1712],
        [0.1988, 0.2274, 0.1785, 0.2248, 0.1705],


Outputs of prediction are 5 dimensional vector. This is so because we have used 5 neurons in the output layer and our activation function is softmax. The 5 dimensional output vector for an input add to 1. So it can be interpreted as probability. Thus we should classify the input to a class, for which prediction probability is maximum. To get the class corresponding to maximum probability, we can use `np.argmax()` or `torch.argmax()` command.

In [41]:
torch.argmax(preds, dim = 1)

tensor([1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 1, 1, 3, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1])

Remember that our data are randomly generated. So we should not be surprised by this result.

This brings us to the end of the blog. As we had planned in the beginning, we have created random data files, a custom dataloader, trained a model using that dataloader, and made predictions on new data. The above code can be tweaked slightly to read any type of files other than .csv. And now we can train our model without worrying about the data size. Whether the data size is 10GB or 750GB, our approach will work for both.

Also note that we have not used the multi-worker and multi-processing capabilities of dataloader. To further speedup the dataloading process, readers should take advantage of the multiprocessing capabilities of dataloader. The best way to choose optimum multiprocessing and multi-worker parameters is to try a few ones and see which set of parameters work best for the system under consideration.

As a final note, please keep in mind that the approach we have discussed in only one of many different ways in which we can read multiple files. I have chosen this approach as it seemed natural to me. I have neither strived for efficiency nor elegance. If readers have any better idea, I would be happy to know of it.

I hope this blog would be of help to reader. Please bring any errors or omissions to my notice.

**Disclaimer**: It is very likely that this blog might not have used some of the best practices of PyTorch. This is because the author has a superficial knowledge of PyTorch and is not aware of its best practices. The author (un)fortunately prefers `Tensorflow`. 